In [1]:
using DifferentialEquations
using Plots, LaTeXStrings
using StatsFuns: log1pexp,logtwo

In [2]:
function logcosh(x::Real)
    abs_x = abs(x)
    return abs_x + log1pexp(-2 * abs_x) - logtwo
end

logcosh (generic function with 1 method)

In [3]:
function evolve!(dx,x,p,t)
    """
    Function for solving Differential Equation
    Args:
        t (Float64): Time Step
        x (Vector{Float64}): Shape (n,)

    Returns:
        Output (Vector{Float64}): Shape (n,) 
    """
    w, z, phi, pw, pz, pphi = x
    alpha=p[1]
    dx[1]=dw = pw
    dx[2]=dz = pz
    dx[3]=dphi = L / (mu * w^2)

    dx[4]=dpw = -mu * (-L^2 / (mu^2 * w^3) + M * w / ((w^2 + z^2)^(3 / 2)))
    dx[5]=dpz = -mu * (M * z / ((w^2 + z^2)^(3 / 2)) + alpha * tanh(z / z0))
    dx[6]=dpphi = 0.0

    # return [dw, dz, dphi, dpw, dpz, dpphi]
end

evolve! (generic function with 1 method)

In [4]:
function evolve_jac!(J, x, p, t)
    w, z, phi, pw, pz, pphi = x
    alpha = p[1]
    # Calculate the derivatives
    dhdwdw = -mu * (3 * L^2 / ((mu^2) * w^4) - 3 * M * w^2 / ((w^2 + z^2)^(5/2)) + M / ((w^2 + z^2)^(3/2)))
    dhdwdz = 3 * M * mu * w * z / ((w^2 + z^2)^(5/2))
    dhdzdz = -mu * (-3 * M * z^2 / ((w^2 + z^2)^(5/2)) + M / ((w^2 + z^2)^(3/2)) + alpha / ((cosh(z / z0))^2) / z0)

    J = [
        0.0 0.0 0.0 1.0 0.0 0.0
        0.0 0.0 0.0 0.0 1.0 0.0
        -2*L/(mu*w^3) 0.0 0.0 0.0 0.0 1/(mu*w^2)
        dhdwdw dhdwdz 0.0 0.0 0.0 2*L/(mu*w^3)
        dhdwdz dhdzdz 0.0 0.0 0.0 0.0
        0.0 0.0 0.0 0.0 0.0 0.0
    ]

    nothing
end


evolve_jac! (generic function with 1 method)

In [5]:
function H(mu, ics, alpha)
    """
    Args:
        mu (Float64): Mass of the Test Particle
        ics (Matrix{Float64}): The Phase Space Coordinates of the particle
                        Shape(n,4)

    Returns:
        Energy of the particle (Vector{Float64}): Shape (n,)
    """
    if ndims(ics) == 1
        w, z, phi, pw, pz, pphi = ics[1], ics[2], ics[3], ics[4], ics[5], ics[6]
    else
        w, z, phi, pw, pz, pphi = ics[:, 1], ics[:, 2], ics[:, 3], ics[:, 4], ics[:, 5], ics[:, 6]
    end

    L = pphi

    return mu * (pw^2 / (2 * mu) + pz^2 / (2 * mu) + L^2 / (2 * mu^2 * w^2) - M / sqrt(w^2 + z^2) + alpha * z0 * logcosh(z / z0))
end

H (generic function with 1 method)

In [41]:
mu = 1.0 #! This Value was not given in paper. I solved it 
#! using the constraint on the total energy
M = 1.0
L = 1.0
z0 = 0.5
alpha = 0.01  #TODO Important Parameter. 
p = [alpha];

In [ ]:
ics = [2.5, 0.0, 0.0, 0.0, 0.49, L]
t = 0:1:100000;
t_span = (0, last(t));

In [ ]:
f! = ODEFunction(evolve!, jac=evolve_jac!)
prob = ODEProblem(f!, ics, t_span, p);

In [ ]:
# Define the callback
poincare(u, t, integrator) = u[2]  # Trigger when the second component becomes zero
affect!(integrator) = integrator.u[2] = 0.0  # Reset the second component to zero
cb = ContinuousCallback(poincare, affect!);

In [ ]:
sol = solve(prob,reltol = 1e-11, abstol=1e-11, callback=cb, maxiters=1e6);

In [ ]:
# energy=[H(mu,sol.u[i],alpha) for i in 1:length(sol.u)];
# rel_e=(energy.-energy[1])/energy[1];
# plot(rel_e, label="Relative Energy Difference")
# xlabel!("Time Step")
# ylabel!("Relative Energy")
# title!("Relative Energy Difference vs. Time Step")

In [ ]:
poi_points=filter(x -> x[5] >= 0 && x[2] == 0, sol.u);

In [ ]:
w = [v[1] for v in poi_points];
vw = [v[4] for v in poi_points];

In [ ]:
plot(w,vw, seriestype=:scatter,ms=0.5, legend=false)
xlabel!("w")
ylabel!(L"v_z")

# Add a title with LaTeX
title!(latexstring("\\mathrm{Poincare\\, \\, Plot\\, \\, when\\, z=0\\, and\\, v_z>0\\,}"))